In [7]:
#Data Cleaning Phase 
!pip install pandas numpy matplotlib seaborn --quit



Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: --quit


In [16]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [14]:
#load dataser from Azure ML
from azureml.core import Workspace, Dataset

ws = Workspace.from_config()
dataset_name = 'synthetic_surgical_risk_data'

try:
    dataset = ws.datasets[dataset_name]
    df = dataset.to_pandas_dataframe()
    print(f"✅ Loaded dataset: '{dataset_name}'")
    print(f"   Shape: {df.shape} (rows, columns)")
    print(f"   Columns: {list(df.columns)}")
except Exception as e:
    print(f"❌ ERROR: {e}")
    print("   Make sure you completed Phase 1 and the dataset is registered.")

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}
✅ Loaded dataset: 'synthetic_surgical_risk_data'
   Shape: (500, 10) (rows, columns)
   Columns: ['patient_id', 'age', 'bmi', 'blood_pressure_systolic', 'cholesterol', 'has_diabetes', 'has_hypertension', 'prior_surgeries', 'asa_score', 'high_risk_label']


Class DeploymentTemplateOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/mlflow/__init__.py:41: UserWarning: Versions of mlflow (3.8.1) and child packages mlflow-skinny (3.5.0) are different. This may lead to unexpected behavior. Please install the same version of all MLflow packages.
  mlflow.mismatch._check_version_mismatch()
Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


In [17]:
df.head()

patient_id  age   bmi  blood_pressure_systolic  cholesterol  has_diabetes  \
0           1   69  26.6                      105          198             0   
1           2   32  20.3                      139          243             1   
2           3   89  35.2                      111          173             0   
3           4   78  30.5                      152          297             0   
4           5   38  27.6                      170          186             0   

   has_hypertension  prior_surgeries  asa_score  high_risk_label  
0                 0                0          2                1  
1                 1                1          3                0  
2                 0                2          3                0  
3                 1                1          3                0  
4                 0                4          2                0

In [19]:
#check for missing values
df.isnull().sum()

patient_id                 0
age                        0
bmi                        0
blood_pressure_systolic    0
cholesterol                0
has_diabetes               0
has_hypertension           0
prior_surgeries            0
asa_score                  0
high_risk_label            0
dtype: int64

In [20]:
#there are no missing values . We move onto checking for implausible medical values. This means that we're checking for observations that are not believable 
reasonable_ranges = {
    'age': (18, 120),
    'bmi': (10, 60),
    'blood_pressure_systolic': (70, 250),
    'cholesterol': (100, 400),
    'prior_surgeries': (0, 20)
}

#check each features 
for column, (min_val, max_val) in reasonable_ranges.items():
    if column in df.columns:
        below_min = (df[column] < min_val).sum()
        above_max = (df[column] > max_val).sum()
        print(f"  {column}: {below_min} values < {min_val}, {above_max} values > {max_val}")

  age: 0 values < 18, 0 values > 120
  bmi: 0 values < 10, 0 values > 60
  blood_pressure_systolic: 0 values < 70, 0 values > 250
  cholesterol: 0 values < 100, 0 values > 400
  prior_surgeries: 0 values < 0, 0 values > 20


In [21]:
#checking for duplicate patient records
# Check for duplicate rows
print("\n🔍 Checking for duplicate rows:")
duplicate_count = df.duplicated().sum()
print(f"  Found {duplicate_count} exact duplicate rows")

# Check for duplicate patient IDs (should be unique)
if 'patient_id' in df.columns:
    duplicate_ids = df['patient_id'].duplicated().sum()
    print(f"  Found {duplicate_ids} duplicate patient IDs")


🔍 Checking for duplicate rows:
  Found 0 exact duplicate rows
  Found 0 duplicate patient IDs


In [22]:
# Check data types and basic stats
print("\n🔍 Data types:")
print(df.dtypes)

print("\n📊 Basic statistics:")
print(df.describe())


🔍 Data types:
patient_id                   int64
age                          int64
bmi                        float64
blood_pressure_systolic      int64
cholesterol                  int64
has_diabetes                 int64
has_hypertension             int64
prior_surgeries              int64
asa_score                    int64
high_risk_label              int64
dtype: object

📊 Basic statistics:
       patient_id         age         bmi  blood_pressure_systolic  \
count  500.000000  500.000000  500.000000               500.000000   
mean   250.500000   52.930000   28.949600               139.352000   
std    144.481833   21.009519    6.260051                23.042961   
min      1.000000   18.000000   18.600000               100.000000   
25%    125.750000   34.000000   23.300000               119.000000   
50%    250.500000   52.000000   28.650000               139.000000   
75%    375.250000   71.000000   34.425000               159.000000   
max    500.000000   89.000000   40.00000